# Mandatory Assignment 2

#### Part II: BERT


***
Here you should build a BERT classifier for the same dataset, banking77, that you used in Part 1 (Bog of Words). You should report results using a classification report. Also, include some reflections on your results: how do they compare with the results from Part I, BoW? Are there any hyperparameters that are particularly important.

You should follow the steps given in the bert_guide notebook, which are 

1. Activate GPU (If available)
2. Load data
3. Inspect data
4. BERT Tokenizer
5. BERT classifier
6. Plot loss and accuracy
7. Reflections


***